In [ ]:
import pandas as pd
import spacy
import datetime

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

Found GPU at: /device:GPU:0
name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 418.67, 16280 MiB


In [ ]:
import pandas as pd
import numpy as np
import os

import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing
import datetime

import logging  # Setting up the loggings to monitor gensim

In [ ]:
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_temp = pd.read_csv('/content/drive/My Drive/capstone_data/data/5_domestic_filter_Ngram_stopwords_lemmatize/HK_domestic_Ngram_stopword_lematize.csv',index_col=[0])

In [ ]:
df_temp['article_text_Ngram_stopword_lemmatize']

0       NEWS Lodro Rinzler second senior Shambhala tea...
1       Lavinia Ideas fleeting likeair molecule flicks...
4       Unnecessary Worry In darkness many ditch obsta...
5       Everything change thoughtsare sound move like ...
8       NEWS Passing World renowned Scholar Jao Tsung ...
                              ...                        
5036    Unsupervised machine learn could one thing giv...
5039    Majority educator Asia Pacific 95 percent reco...
5051    Launched January 2016 Saensuk 's smart healthc...
5055    Asia 's Premier Business Technology Publicatio...
5060    Asia 's Premier Business Technology Publicatio...
Name: article_text_Ngram_stopword_lemmatize, Length: 2302, dtype: object

In [ ]:
#Import the whole textfile for the dataframe

directory = '/content/drive/My Drive/capstone_data/data/5_domestic_filter_Ngram_stopwords_lemmatize'

df_dict = {'country':[], 'text':[]}
for entry_country in os.scandir(directory):
  path = entry_country.path
  country = entry_country.name.split('_domestic')[0]
  df = pd.read_csv(path, index_col = 0)
  print(country,df.shape[0],end=', ',flush=True)
  df_dict['country'] += [country for _ in range(df.shape[0])]
  df_dict['text'] += list(df['article_text_Ngram_stopword_lemmatize'])

df_for_w2v = pd.DataFrame.from_dict(df_dict)

# nine_countries_list = ['CA', 'NZ', 'SG', 'LK', 'IN', 'PH', 'PK', 'BD', 'NG']

# df_for_w2v = df_for_w2v[df_for_w2v['country'].isin(nine_countries_list)]
# df_for_w2v = df_for_w2v.dropna()

df_for_w2v.head()

HK 2302, LK 11330, SG 20196, TZ 6165, KE 30695, JM 33402, MY 30410, PH 61518, NG 52896, NZ 56484, CA 73870, GH 22784, IE 60823, AU 62684, IN 76556, BD 15246, US 67406, GB 39514, ZA 64428, PK 69960, 

,country,text
0,HK,NEWS Lodro Rinzler second senior Shambhala tea...
1,HK,Lavinia Ideas fleeting likeair molecule flicks...
2,HK,Unnecessary Worry In darkness many ditch obsta...
3,HK,Everything change thoughtsare sound move like ...
4,HK,NEWS Passing World renowned Scholar Jao Tsung ...


In [ ]:
df_for_w2v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858669 entries, 0 to 858668
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   country  858669 non-null  object
 1   text     857978 non-null  object
dtypes: object(2)
memory usage: 13.1+ MB


In [ ]:
df_for_w2v = df_for_w2v.groupby('country').apply(pd.DataFrame.sample, frac=0.75).reset_index(drop=True)
df_for_w2v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644001 entries, 0 to 644000
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   country  644001 non-null  object
 1   text     643492 non-null  object
dtypes: object(2)
memory usage: 9.8+ MB


In [ ]:
df_for_w2v = df_for_w2v.dropna()
df_for_w2v.info()

INFO - 08:39:19: NumExpr defaulting to 4 threads.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 643492 entries, 0 to 644000
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   country  643492 non-null  object
 1   text     643492 non-null  object
dtypes: object(2)
memory usage: 14.7+ MB


In [ ]:
import multiprocessing

from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

#
sent = [row.split() for row in df_for_w2v['text']]

#
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 08:39:20: 'pattern' package not found; tag filters are not available for English
INFO - 08:39:55: collecting all words and their counts
INFO - 08:39:55: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 08:40:03: PROGRESS: at sentence #10000, processed 4113326 words and 2348946 word types
INFO - 08:40:12: PROGRESS: at sentence #20000, processed 8239552 words and 4090708 word types
INFO - 08:40:21: PROGRESS: at sentence #30000, processed 12411103 words and 5621991 word types
INFO - 08:40:30: PROGRESS: at sentence #40000, processed 16520486 words and 7004915 word types
INFO - 08:40:38: PROGRESS: at sentence #50000, processed 20403265 words and 8294294 word types
INFO - 08:40:45: PROGRESS: at sentence #60000, processed 23820525 words and 9375737 word types
INFO - 08:40:55: PROGRESS: at sentence #70000, processed 28222778 words and 10783030 word types
INFO - 08:41:06: PROGRESS: at sentence #80000, processed 32566398 words and 12063357 word types
INFO - 08:41:15: PR

In [ ]:
bigram = Phraser(phrases)

INFO - 08:49:37: source_vocab length 31958540
INFO - 08:50:26: Phraser added 50000 phrasegrams
INFO - 08:55:40: Phraser built with 92362 phrasegrams


In [ ]:
sentences = bigram[sent]

In [ ]:
# word_freq = defaultdict(int)
# for sent in sentences:
#     for i in sent:
#         word_freq[i] += 1
# len(word_freq)

In [ ]:
# sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
cores = multiprocessing.cpu_count()

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 08:55:40: collecting all words and their counts
INFO - 08:55:40: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 08:55:52: PROGRESS: at sentence #10000, processed 3846725 words, keeping 149660 word types
INFO - 08:56:04: PROGRESS: at sentence #20000, processed 7703229 words, keeping 213830 word types
INFO - 08:56:16: PROGRESS: at sentence #30000, processed 11601465 words, keeping 262359 word types
INFO - 08:56:28: PROGRESS: at sentence #40000, processed 15438142 words, keeping 304570 word types
INFO - 08:56:39: PROGRESS: at sentence #50000, processed 19056674 words, keeping 349157 word types
INFO - 08:56:49: PROGRESS: at sentence #60000, processed 22212013 words, keeping 389719 word types
INFO - 08:57:02: PROGRESS: at sentence #70000, processed 26287348 words, keeping 436504 word types
INFO - 08:57:15: PROGRESS: at sentence #80000, processed 30309477 words, keeping 474379 word types
INFO - 08:57:27: PROGRESS: at sentence #90000, processed 34291744 words,

Time to build vocab: 12.65 mins


In [ ]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
INFO - 14:25:06: EPOCH 16 - PROGRESS: at 74.33% examples, 122670 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:07: EPOCH 16 - PROGRESS: at 74.41% examples, 122666 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:08: EPOCH 16 - PROGRESS: at 74.50% examples, 122674 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:09: EPOCH 16 - PROGRESS: at 74.58% examples, 122670 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:10: EPOCH 16 - PROGRESS: at 74.66% examples, 122665 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:11: EPOCH 16 - PROGRESS: at 74.75% examples, 122666 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:12: EPOCH 16 - PROGRESS: at 74.83% examples, 122672 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:13: EPOCH 16 - PROGRESS: at 74.92% examples, 122668 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:14: EPOCH 16 - PROGRESS: at 75.00% examples, 122671 words/s, in_qsize 0, out_qsize 0
INFO - 14:25:15: EPOCH 16 - PROGRESS: at 75.10% examples, 122667 words/s, in_qsize

Time to train the model: 402.46 mins


In [ ]:
w2v_model.init_sims(replace=True)

INFO - 15:50:47: precomputing L2-norms of word weight vectors


In [ ]:
w2v_model.save("/content/drive/My Drive/word2vec_20_countries_new_75.model")

INFO - 15:50:51: saving Word2Vec object under /content/drive/My Drive/word2vec_20_countries_new_75.model, separately None
INFO - 15:50:51: storing np array 'vectors' to /content/drive/My Drive/word2vec_20_countries_new_75.model.wv.vectors.npy
INFO - 15:50:52: not storing attribute vectors_norm
INFO - 15:50:52: storing np array 'syn1neg' to /content/drive/My Drive/word2vec_20_countries_new_75.model.trainables.syn1neg.npy
INFO - 15:50:53: not storing attribute cum_table
INFO - 15:50:54: saved /content/drive/My Drive/word2vec_20_countries_new_75.model


In [ ]:
w2v_model.wv.most_similar(positive=["government"])